##Импортирование библиотек

In [62]:
!pip install datasets
!pip install accelerate
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score

Задаем seed

In [29]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

##Получение данных из датасета

Получаем данные из всех датасетов(train,dev,test)
Создаем массивы

*   all_ids - список всех id
*   sentences - список всех симтомов
*   labels - список всех меток                

In [49]:
import json

parts = ['train', 'dev', 'test']

part2indices = {p:set() for p in parts}
all_ids, symptoms, labels = [], [], []

for p in parts:
    fname = '{}_v1.jsonl'.format(p)
    with open(f"data/{fname}") as f:
        data_list = []
        for line in f:
            data = json.loads(line)
            symptoms.append(data['symptoms'])
            labels.append(data['code'])
            idx = data['idx']
            all_ids.append(idx)
            part2indices[p].add(idx)
            data_list.append(data)

        # Создаем датафрейм для текущей части
        df = pd.DataFrame(data_list)

        # Сохраняем датафрейм в соответствующую переменную
        if p == 'train':
            train_df = df
        elif p == 'dev':
            dev_df = df
        elif p == 'test':
            test_df = df

train_text = train_df['symptoms'].astype('str')
train_labels = train_df['code']
test_text = test_df['symptoms'].astype('str')
test_labels = test_df['code']
dev_text = test_df['symptoms'].astype('str')
dev_labels = test_df['code']

all_ids = np.array(all_ids)
print ('len(total)', len(symptoms))

# Выводим по 10 значений из каждого массива
print('First 10 sentences:')
print(symptoms[:10])
print('First 10 labels:')
print(labels[:10])
print('First 10 ids:')
print(all_ids[:10])

len(total) 6360
First 10 sentences:
['Жалобы на болит во всех суставах но более в крупных суставах, в тенчние 2-х недель, подобное состояние было месяц назад полсе стресса на работае, все купировалось в покое, нарушен сон.', 'На боли в правом плече с иррадиацией в локтевой сустав.', 'боль в эпигастрии после погрешности в диете, внизу живота. Стул, диурез, температура в норме.', 'боль в правом плечевом суставе, в правом плече, с иррад в кисть, подергивание и онемение в правой руке. Болевой синдром на фоне терапии (каликсат, катэна) менее выражен, увеличен объем движений в правом плечевом суставе.', '- приливы пота в только на улице - снижение веса на фоне сохраненного аппетита - АД в основном 100/70, при волнении до 130/. - бессонница.', 'на учащение приступов головной боли', 'на головную боль, заложенность носа, насморк, темп тела в норме', 'Жалобы на головные боли, головокружение, слабость, перепады АД и нарушение ритма. Осмотрена кардиологом и неврологом, офтальмологом. Рекомендовано

создаем словари l2i и i2l (code to id и id to code соответственно)

In [44]:
code_set = set(labels)
l2i = {code:i for i, code in enumerate(sorted(code_set))}
i2l = {l2i[l]:l for l in l2i}
print ( 'len(l2i)', len(l2i) )

# Выводим первые 10 записей из словаря l2i
print('First 10 items in l2i:')
for code, index in list(l2i.items())[:10]:
    print(f'Code: {code}, Index: {index}')

len(l2i) 105
First 10 items in l2i:
Code: D23, Index: 0
Code: D25, Index: 1
Code: D50, Index: 2
Code: E01, Index: 3
Code: E03, Index: 4
Code: E04, Index: 5
Code: E05, Index: 6
Code: E06, Index: 7
Code: E11, Index: 8
Code: E22, Index: 9


Далее происходит загрузка предобученной модели, а также указание, на какое количество классов делятся данные. Для дообучения будет использовать GPU:

In [35]:
tokenizer = AutoTokenizer.from_pretrained("alexyalunin/RuBioRoBERTa")
model = AutoModelForMaskedLM.from_pretrained("alexyalunin/RuBioRoBERTa", num_labels=len(l2i)).to("cuda")

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Проверка максимальной длины предложения

In [47]:
seq_len_train = [len(str(i).split()) for i in train_df['symptoms']]
seq_len_test = [len(str(i).split()) for i in test_df['symptoms']]
seq_len_dev = [len(str(i).split()) for i in dev_df['symptoms']]
max_seq_len = max(max(seq_len_test), max(seq_len_train), max(seq_len_dev))
max_seq_len

531

*Решить что делать с длиной входных данных(531 > 512)*

In [51]:
tokens_train = tokenizer.batch_encode_plus(
    list(train_text.values),
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)
tokens_test = tokenizer.batch_encode_plus(
    list(test_text.values),
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)
tokens_dev = tokenizer.batch_encode_plus(
    list(test_text.values),
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)

Данный код оборачивает токенизированные текстовые данные в torch Dataset:

In [52]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = Data(tokens_train, train_labels)
test_dataset = Data(tokens_test, test_labels)
dev_dataset = Data(tokens_dev, dev_labels)

Корректость работы нейросети будем проверять при помощи метрики F1, так как классы не сбалансированны

In [53]:
from sklearn.metrics import f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {'F1': f1}

Ниже указаны все параметры, которые будут использоваться для обучения:

In [ ]:
!pip install accelerate -U

In [68]:
!python -c "TODO write"

  File "<string>", line 1
    TODO write
         ^^^^^
SyntaxError: invalid syntax


In [66]:
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 3, #Кол-во эпох для обучения
    per_device_train_batch_size = 8, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=21)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

Передача в trainer предообученную модель, tokenizer, данные для обучения, данные для валидации и способ расчета метрики:

In [ ]:
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = train_dataset,
                  compute_metrics = compute_metrics)

Запуск обучения модели:

In [ ]:
trainer.train()